In [ ]:
import math
import random
import time
# Global variables
n = 3 # set up size for board game
rows = []
cols = []
diagonal = 0
anti_diagonal = 0
size = 0
MAX_DEPTH = 4  # Limiting the depth for larger boards
# set up the target_consecutive for checking winner
target_consecutive = 3 if n<=4 else 5


In [ ]:
# initialize the grid in 2D rows and columns for the game
def initialize_grid(n):
    global rows, cols, diagonal, anti_diagonal, size
    size = n
    rows = [0 for _ in range(n)]
    cols = [0 for _ in range(n)]
    diagonal = anti_diagonal = 0
    return [[" " for _ in range(n)] for _ in range(n)]

In [ ]:
# this function support the make_move function for the AI moves
def update_counters(grid, posX, posY, value):
    global rows, cols, diagonal, anti_diagonal, size

    # Update rows
    if all(grid[posX][i] == value for i in range(posY, min(posY + target_consecutive, size))):
        rows[posX] = value
    else:
        rows[posX] = None

    # Update columns
    if all(grid[i][posY] == value for i in range(posX, min(posX + target_consecutive, size))):
        cols[posY] = value
    else:
        cols[posY] = None

    if posX == posY:
        if all(grid[i][i] == value for i in range(max(0, posX - target_consecutive + 1), min(size, posX + target_consecutive))):
            diagonal = value
        else:
            diagonal = None

    # Update anti-diagonal if the move is on the anti-diagonal
    if posX + posY == size - 1:
        if all(grid[i][size - 1 - i] == value for i in range(max(0, posX - target_consecutive + 1), min(size, posX + target_consecutive))):
            anti_diagonal = value
        else:
            anti_diagonal = None


In [ ]:
# Update the moves on the grid
def make_move(grid, posX, posY,player):
    if 0 <= posX < size and 0 <= posY < size and grid[posX][posY] == " ":
        grid[posX][posY] = player
        value = 1 if player == 'X' else -1
        update_counters(grid, posX, posY, value)
    else:
        raise ValueError("Invalid move")


In [ ]:
# the function determines the winner by checking any rows, columns, diagonal, anti-diagonal reach consecutive-target symbol
def check(grid, row, col, target):
    player_symbol = grid[row][col]
    n = len(grid)
    # function to check the direction of moves for both player and AI (this is idea based on the bfs technique)
    def check_direction(dx, dy):
        count = 0
        x, y = row, col
        while 0 <= x < n and 0 <= y < n and grid[x][y] == player_symbol:
            count += 1
            x += dx
            y += dy
        return count

    # Those code lines shows the number of symbols to determine or check if the count is greater than the consecutive target for win the game
    # Check row
    row_count = check_direction(0, -1) + check_direction(0, 1) - 1  # -1 to avoid double counting the last move
    if row_count >= target:
        return player_symbol

    # Check column
    col_count = check_direction(-1, 0) + check_direction(1, 0) - 1
    if col_count >= target:
        return player_symbol

    # Check main diagonal
    if row == col:
        diag_count = check_direction(-1, -1) + check_direction(1, 1) - 1
        if diag_count >= target:
            return player_symbol

    # Check anti-diagonal
    if row + col == n - 1:
        anti_diag_count = check_direction(-1, 1) + check_direction(1, -1) - 1
        if anti_diag_count >= target:
            return player_symbol

    return None



In [ ]:
########################################
"""
    This functions for minimax approach
"""


# set up heuristic function for AI move to find optimals
def heuristic_evaluation(grid):
    score = 0

    # Evaluate all rows, columns, and diagonals
    for i in range(size):
        score += evaluate_line([grid[i][j] for j in range(size)])  # Row
        score += evaluate_line([grid[j][i] for j in range(size)])  # Column

    score += evaluate_line([grid[i][i] for i in range(size)])  # Main diagonal
    score += evaluate_line([grid[i][size - i - 1] for i in range(size)])  # Anti-diagonal

    return score

In [ ]:
# this function add-on to support AI become more intelligent to find the best way to win game or block the player if player nearly win the game
def evaluate_line(line):
    x_count = line.count('X')
    o_count = line.count('O')
    empty_count = line.count(' ')

    #update score for win game by finding optimal the moves
    if x_count == target_consecutive:
        return 1000
    elif o_count == target_consecutive:
        return -1000
    #blocking player
    elif x_count == target_consecutive - 1 and empty_count == 1:
        return 500
    elif o_count == target_consecutive - 1 and empty_count == 1:
        return -500
    else:
        return x_count - o_count


In [ ]:
#This function to check if game is over or not by returning the True or False, if true then the game continue otherwise break the game
def is_game_over(grid):
    for player in ['X', 'O']:
        if any(all(grid[i][j] == player for j in range(size)) for i in range(size)) or \
           any(all(grid[j][i] == player for j in range(size)) for i in range(size)) or \
           all(grid[i][i] == player for i in range(size)) or \
           all(grid[i][size - 1 - i] == player for i in range(size)):
            return True

    if all(grid[i][j] != " " for i in range(size) for j in range(size)):
        return True

    return False


In [ ]:
# function to predict best moves
def get_possible_moves(grid):
    possible_moves = []
    for i in range(size):
        for j in range(size):
            if grid[i][j] == " ":
                possible_moves.append((i, j))
    return possible_moves

In [ ]:
# clear the move after tries if not find the best
def undo_move(grid, move):
    row, col = move
    grid[row][col] = " "

In [ ]:
# set up depth for minimax algorithm
def dynamic_minimax_depth(empty_spaces):
    if empty_spaces > (size * size) // 2:
        return 2  # Early game, shallower depth
    else:
        return MAX_DEPTH  # Deeper search as the game progresses

In [ ]:
#minimax function with alpha-beta-pruning optimization
def  minimax(grid, depth, isMaximizingPlayer, alpha, beta, empty_spaces=None, last_move=None):
    # Base case: Evaluate the board
    if depth == 0 or is_game_over(grid):
        return heuristic_evaluation(grid)

    if isMaximizingPlayer:
        maxEval = -math.inf
        for move in get_possible_moves(grid):
            make_move(grid, move[0], move[1], 'X')
            eval = minimax(grid, depth - 1, False, alpha, beta)
            undo_move(grid, move)
            maxEval = max(maxEval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return maxEval
    else:
        minEval = math.inf
        for move in get_possible_moves(grid):
            make_move(grid, move[0], move[1], 'O')
            eval = minimax(grid, depth - 1, True, alpha, beta)
            undo_move(grid, move)
            minEval = min(minEval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return minEval


In [ ]:
def find_best_move(grid):
    bestVal = -math.inf
    bestMove = (-1, -1)
    empty_spaces = sum(row.count(' ') for row in grid)
    current_depth = MAX_DEPTH if empty_spaces > 6 else MAX_DEPTH + 1  # Count empty spaces at the beginning

    for i in range(size):
        for j in range(size):
            if grid[i][j] == " ":
                # Simulate the move
                grid[i][j] = 'X'
                update_counters(grid, i, j, 1)

                # Call minimax using the simulated move as the last move
                empty_spaces = sum(row.count(' ') for row in grid)  # Count the empty spaces
                last_move = (i, j)  # Assuming i and j are the row and column of the last move
                current_depth = dynamic_minimax_depth(empty_spaces)
                moveVal = minimax(grid, current_depth, False, -math.inf, math.inf, empty_spaces, last_move)

                # Undo the simulated move
                grid[i][j] = " "
                update_counters(grid, i, j, -1)

                # Update best value and move
                if moveVal > bestVal:
                    bestMove = (i, j)
                    bestVal = moveVal

    return bestMove

In [ ]:
"""
    This functions for IDDFS approach
"""

def iddfs(grid, max_depth):
    best_move = (-1, -1)
    best_value = -math.inf

    for depth in range(1, max_depth + 1):
        value, move = dfs(grid, depth, True, -math.inf, math.inf)
        if value > best_value:
            best_value = value
            best_move = move

    return best_move

def dfs(grid, depth, is_maximizing_player, alpha, beta):
    if depth == 0 or is_game_over(grid):
        return heuristic_evaluation(grid), None

    if is_maximizing_player:
        max_value = -math.inf
        best_move = None
        for move in get_possible_moves(grid):
            make_move(grid, move[0], move[1], 'X')
            value, _ = dfs(grid, depth - 1, False, alpha, beta)
            undo_move(grid, move)

            if value > max_value:
                max_value = value
                best_move = move

            alpha = max(alpha, value)
            if beta <= alpha:
                break

        return max_value, best_move

    else:
        min_value = math.inf
        best_move = None
        for move in get_possible_moves(grid):
            make_move(grid, move[0], move[1], 'O')
            value, _ = dfs(grid, depth - 1, True, alpha, beta)
            undo_move(grid, move)

            if value < min_value:
                min_value = value
                best_move = move

            beta = min(beta, value)
            if beta <= alpha:
                break

        return min_value, best_move



def find_best_move_iddfs(grid):
    empty_spaces = sum(row.count(' ') for row in grid)
    max_depth = dynamic_minimax_depth(empty_spaces)
    return iddfs(grid, max_depth)



In [ ]:
def hillclimbing(grid, firstmove):
    """first move of hill climbing is random"""
    if firstmove:
        moves = get_possible_moves(grid)
        return random.choice(moves)

    """convert grid into numbers to calculate steepest ascent"""
    hillgrid = [[0] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            if grid[i][j] == 'X':
                hillgrid[i][j] = 1
            if grid[i][j] == 'O':
                hillgrid[i][j] = -100

    totals = sumAll(hillgrid)

    """perform action according to calculation"""
    if totals[1] == 'horz':
        for i in range(n):
            if hillgrid[totals[0]][i] == 0:
                return (totals[0], i)
            if i == n-1:
                break
    elif totals[1] == 'vert':
        for i in range(n):
            if hillgrid[i][totals[0]] == 0:
                return (i, totals[0])
    elif totals[1] == 'diag':
        for i in range(n):
            if hillgrid[i][i] == 0:
                return(i, i)
    elif totals[1] == 'adiag':
        for i in range(n):
            if hillgrid[i][n-i-1] == 0:
                return(i, n-i-1)

    for i in range(n):
        for j in range(n):
            if grid[i][j] == ' ':
                return (i,j)



"""function to find the row/col/diag with the best possibility to win"""
def sumAll(hgrid):
    map = {'horz': [0, -1], 'vert': [0, -1], 'diag': [0, -1], 'adiag': [0, -1]}


    hMax = -1000
    vMax = -1000
    h = 0
    v = 0

    for i in range(n):
        for j in range(n):
            h += hgrid[i][j]
            v += hgrid[j][i]
        if h > hMax:
            hMax = h
            map['horz'][1] = i
            map['horz'][0] = h
        if v > vMax:
            vMax = v
            map['vert'][1] = i
            map['vert'][0] = v
        h = 0
        v = 0

    for i in range(n):
        map['diag'][0] += hgrid[i][i]
        map['adiag'][0] += hgrid[i][n-i-1]

    high = -999
    answer = ()
    for key,val in map.items():
        print(val[0])
        if val[0] > high:
            answer = (val[1], key)
            high = val[0]

    return answer



In [ ]:

########################################
"""
    This functions for print out
"""


def print_grid(grid, n):
    """Prints the tic-tac-toe grid."""
    for row in range(len(grid)):
        if row == 0:
           print("-" * (n+ (n+1)))
        print("|"+"|".join(grid[row]) + "|")
        print("-" * (n+ (n+1)))


def player_input(grid):
    while True:
        try:
            input_str = input("Enter your move (row col): ")
            row, col = map(int, input_str.split())  # Convert to integers
            if 0 <= row < size and 0 <= col < size and grid[row][col] == " ":
                return row, col
            else:
                print("Invalid move. Please try again.")
        except ValueError:
            print("Invalid input. Please enter row and column numbers.")

In [ ]:

########################################

def play_game():
    print("1: Minimax and optimize with alpha-beta-pruning algorithm \n2: IDDFS algorithm \n3: Hill Climbing")
    global grid,target_consecutive
    player_symbol = 'O'  # Assuming the player is 'O'
    ai_symbol = 'X'     # Assuming the AI is 'X'
      # Number of consecutive symbols needed to win
    x=int(input("Choose algorithm: "))
    if x==1:
        print("Minimax algorithm")
        while True:
            # Player's turn
            print_grid(grid, n)
            row, col = player_input(grid)
            make_move(grid, row, col,player_symbol)
            print_grid(grid, n)

            winner = check(grid, row,col, target_consecutive)
            if winner:
                print(f"Player {winner} wins!")
                break

            if all(grid[i][j] != " " for i in range(size) for j in range(size)):
                print("The game is a draw.")
                break

            # AI's turn
            print("AI is making a move...")
            start_time=time.time()
            best_move = find_best_move(grid)
            make_move( grid, best_move[0], best_move[1],ai_symbol)

            winner = check(grid, best_move[0], best_move[1], target_consecutive)
            end_time=time.time()
            print(f"AI's move took {end_time - start_time} seconds.")
            if winner:
                print_grid(grid, n)
                print(f"Player {winner} wins!")
                break
    elif x==2:
        print("IDDFS algorithm")
        while True:
            # Player's turn
            print_grid(grid, n)
            row, col = player_input(grid)
            make_move(grid, row, col, player_symbol)
            print_grid(grid, n)

            winner = check(grid, row, col, target_consecutive)
            if winner:
                print(f"Player {winner} wins!")
                break

            if all(grid[i][j] != " " for i in range(size) for j in range(size)):
                print("The game is a draw.")
                break

            # AI's turn
            print("AI is making a move...")
            start_time = time.time()
            best_move = find_best_move_iddfs(grid)  # Use IDDFS for AI's decision making
            make_move(grid, best_move[0], best_move[1], ai_symbol)

            winner = check(grid, best_move[0], best_move[1], target_consecutive)
            end_time = time.time()
            print(f"AI's move took {end_time - start_time} seconds.")
            if winner:
                print_grid(grid, n)
                print(f"Player {winner} wins!")
                break
    elif x==3:
        print("Hill-climbing algorithm")
        firstmove = True
        while True:
            # Player's turn
            print_grid(grid, n)
            row, col = player_input(grid)
            make_move(grid, row, col, player_symbol)
            print_grid(grid, n)

            winner = check(grid, row, col, target_consecutive)
            if winner:
                print(f"Player {winner} wins!")
                break

            if all(grid[i][j] != " " for i in range(size) for j in range(size)):
                print("The game is a draw.")
                break

            # AI's turn
            print("AI is making a move...")
            start_time = time.time()
            best_move = hillclimbing(grid, firstmove) # Use IDDFS for AI's decision making
            make_move(grid, best_move[0], best_move[1], ai_symbol)
            firstmove = False
            winner = check(grid, best_move[0], best_move[1], target_consecutive)
            end_time = time.time()
            print(f"AI's move took {end_time - start_time} seconds.")
            if winner:
                print_grid(grid, n)
                print(f"Player {winner} wins!")
                break


In [ ]:
# Initialize and start the game

grid = initialize_grid(n)
play_game()

1: Minimax and optimize with alpha-beta-pruning algorithm 
2: IDDFS algorithm 
3: Hill Climbing
Choose algorithm: 3
Hill-climbing algorithm
-------
| | | |
-------
| | | |
-------
| | | |
-------
Enter your move (row col): 0 1
-------
| |O| |
-------
| | | |
-------
| | | |
-------
AI is making a move...
AI's move took 5.1975250244140625e-05 seconds.
-------
| |O| |
-------
| | | |
-------
| |X| |
-------
Enter your move (row col): 0 0
-------
|O|O| |
-------
| | | |
-------
| |X| |
-------
AI is making a move...
1
0
-100
0
AI's move took 0.004278421401977539 seconds.
-------
|O|O| |
-------
| | | |
-------
|X|X| |
-------
Enter your move (row col): 0 2
-------
|O|O|O|
-------
| | | |
-------
|X|X| |
-------
Player O wins!
